<p><font size='6' face='Times New Roman'><b><u>Generic Transportation Problem Example</u></b></font></p>

<p><font size='5' face='Times New Roman'><b><i>James Gaboardi, 2015</i></b></font></p>

--------------------

<p><font size='6' face='Times New Roman'><u>GNU LESSER GENERAL PUBLIC LICENSE</u></font></p>
<p><font size='3' face='Times New Roman'><b>Version 3, 29 June 2007.</b></font></p>
<p><font size='3' face='Times New Roman'><b>Copyright (C) 2007 Free Software Foundation, Inc. <http://fsf.org/>.</b></font></p>
<p><font size='3' face='Times New Roman'><b>Everyone is permitted to copy and distribute verbatim copies of this license document, but changing it is not allowed.</b></font></p>

-----------------

<font size='7' face='Times New Roman'><b>0. <u>Imports</u></b></font>

In [ ]:
import pysal as ps
import numpy as np
import networkx as nx
import shapefile as shp
import gurobipy as gbp
import time
from collections import OrderedDict
import IPython.display as IPd
%pylab inline

<font size='7' face='Times New Roman'><b>1. <u>Data preparation and creation</u></b></font>

<font size='5' face='Times New Roman'><b>1.1 <u>Instantiate a network</u></b></font>

In [ ]:
ntw = ps.Network('Waverly/Waverly.shp')
print 'PySAL.Network\n'
print dir(ntw)

<font size='5' face='Times New Roman'><b>1.2 <u>Instantiate all graphs to be drawn</u></b></font>

In [ ]:
# Roads and Nodes
g = nx.Graph()
# Graph of Roads and Nodes
g1 = nx.MultiGraph()
# Clients
GRAPH_supply = nx.Graph()
# Snapped Supply
g_supply = nx.Graph()
# Demand
GRAPH_demand = nx.Graph()
# Snapped Demand
g_demand = nx.Graph()

<font size='5' face='Times New Roman'><b>1.3 <u>Create Bounding Box from 'Waverly.shp'</u></b></font>

In [ ]:
shp_W = ps.open('Waverly/Waverly.shp')
shp_W.bbox

<font size='5' face='Times New Roman'><b>1.4 <u>Create numpy arrays of random floats within a  bounding box</u></b></font>

In [ ]:
lat_supply = np.random.uniform(shp_W.bbox[0], shp_W.bbox[2], 100)
lon_supply = np.random.uniform(shp_W.bbox[1], shp_W.bbox[3], 100)
lat_demand = np.random.uniform(shp_W.bbox[0], shp_W.bbox[2], 100)
lon_demand = np.random.uniform(shp_W.bbox[1], shp_W.bbox[3], 100)

<font size='5' face='Times New Roman'><b>1.5 <u>Zip the latitude and longitude lists together</u></b></font>

In [ ]:
rand_coords_supply = map(list, zip(lat_supply, lon_supply))
rand_coords_demand = map(list, zip(lat_demand, lon_demand))

<font size='5' face='Times New Roman'><b>1.6 <u>Create Empty Random Points Dictionaries</u></b></font>

In [ ]:
points_supply = {}
points_demand = {}

<font size='5' face='Times New Roman'><b>1.7 <u>Fill dictionaries of random roints</u></b></font>

In [ ]:
# SUPPLY
for idx, coords in enumerate(rand_coords_supply):
    GRAPH_supply.add_node(idx)
    points_supply[idx] = coords
    GRAPH_supply.node[idx] = coords
# DEMAND   
for idx, coords in enumerate(rand_coords_demand):
    GRAPH_demand.add_node(idx)
    points_demand[idx] = coords
    GRAPH_demand.node[idx] = coords

<font size='5' face='Times New Roman'><b>1.8 <u>Draw graphs of roads, simplified network, and random supply & demand nodes</u></b></font>

In [ ]:
#Instantiate Figure
figsize(10,10)
# Draw Graph of Actual Nodes and Roads
for e in ntw.edges:
    g.add_edge(*e)
nx.draw(g, ntw.node_coords, node_size=5, alpha=0.25, edge_color='r', width=2)
# Draw only unique edges in graph
for e in ntw.graphedges:
    g1.add_edge(*e)
    # highlights cases where start and end node are the same
    if e[0]==e[1]:
        g1.add_node(e[0])
for node_id in g1.node:
    g1.node[node_id] = ntw.node_coords[node_id]
nx.draw(g1, ntw.node_coords, node_size=10, alpha=0.5)
# Draw Graph of Random Supply Points
nx.draw(GRAPH_supply, points_supply, 
    node_size=75, alpha=1, node_color='b')
# Draw Graph of Random Demand Points
nx.draw(GRAPH_demand, points_demand, 
    node_size=100, alpha=1, node_color='c')
# Legend (Ordered Dictionary)
LEGEND = OrderedDict()
LEGEND['Network Nodes']=g
LEGEND['Roads']=g
LEGEND['Graph Vertices']=g1
LEGEND['Graph Edges']=g1
LEGEND['Supply Nodes']=GRAPH_supply
LEGEND['Demand Nodes']=GRAPH_demand
legend(LEGEND, loc='best')
# Title
title('Waverly Hills\n Tallahassee, Florida', family='Times New Roman', 
      size=40, color='k', backgroundcolor='w', weight='bold')

<font size='5' face='Times New Roman'><b>1.9 <u>Create Supply and Demand</u></b></font>

In [ ]:
#Supply
Si = np.random.randint(50, 200, 100)
Si = Si.reshape(100,1)
SiSum = np.sum(Si)
# Demand
Dj = np.sort(Si, axis=None)
DjSum = np.sum(Dj)

<font size='5' face='Times New Roman'><b>1.10 <u>Instantiate supply and demand shapefiles</u></b></font>

In [ ]:
# Supply
supply = shp.Writer(shp.POINT)
# Add Random Points
for i,j in rand_coords_supply:
    supply.point(i,j)
# Add Fields
supply.field('Si_ID')
supply.field('LAT')
supply.field('LON')
supply.field('Supply')
counter = 0
for i in range(len(rand_coords_supply)):
    counter = counter + 1
    supply.record('x'+ str(counter)+'_--', lat_supply[i], lon_supply[i], Si[i])
# Save Shapefile    
supply.save('path.../RandomPoints_SUPPLY')

#Demand
demand = shp.Writer(shp.POINT)
# Add Random Points
for i,j in rand_coords_demand:
    demand.point(i,j)
# Add Fields
demand.field('Dj_ID_x')
demand.field('LAT')
demand.field('LON')
demand.field('Demand')
counter = 0
for i in range(len(rand_coords_demand)):
    counter = counter + 1
    demand.record('x--_'+ str(counter), lat_demand[i], lon_demand[i], Dj[i])
# Save Shapefile    
demand.save('path.../RandomPoints_DEMAND')

<font size='5' face='Times New Roman'><b>1.11 <u>Snap Observations to NTW</u></b></font>

In [ ]:
t1 = time.time()
ntw.snapobservations('shapefiles/RandomPoints_SUPPLY.shp', 
                     'Rand_Points_SUPPLY', attribute=True)
ntw.snapobservations('shapefiles/RandomPoints_DEMAND.shp', 
                     'Rand_Points_DEMAND', attribute=True)
print round(time.time()-t1, 4), 'seconds'

<font size='5' face='Times New Roman'><b>1.12 <u>Draw NTW, snapped coords, & random coords</u></b></font>

In [ ]:
# Instantiate Figure
figsize(10,10)
# Draw Graph of Roads
for e in ntw.edges:
    g.add_edge(*e)
nx.draw(g, ntw.node_coords, node_size=5, alpha=0.25, edge_color='r', width=2)
# Draw Graph of Snapped Supply Nodes
g_supply = nx.Graph()
for p,coords in ntw.pointpatterns['Rand_Points_SUPPLY'].snapped_coordinates.iteritems():
    g_supply.add_node(p)
    g_supply.node[p] = coords
nx.draw(g_supply, ntw.pointpatterns['Rand_Points_SUPPLY'].snapped_coordinates, 
        node_size=100, alpha=1, node_color='b')
# Draw Graph of Snapped Demand Nodes
g_service = nx.Graph()
for p,coords in ntw.pointpatterns['Rand_Points_DEMAND'].snapped_coordinates.iteritems():
    g_demand.add_node(p)
    g_demand.node[p] = coords
nx.draw(g_demand, ntw.pointpatterns['Rand_Points_DEMAND'].snapped_coordinates, 
        node_size=100, alpha=1, node_color='c')
# Draw Graph of Random Supply Points
nx.draw(GRAPH_supply, points_supply, 
    node_size=20, alpha=1, node_color='y')
# Draw Graph of Random Demand Points
nx.draw(GRAPH_demand, points_demand, 
    node_size=20, alpha=1, node_color='w')

# Legend (Ordered Dictionary)
LEGEND = OrderedDict()
LEGEND['Network Nodes']=g
LEGEND['Roads']=g
LEGEND['Snapped Supply']=g_supply
LEGEND['Snapped Demand']=g_demand
LEGEND['Supply Nodes']=GRAPH_supply
LEGEND['Demand Nodes']=GRAPH_demand

legend(LEGEND, loc='best')
# Title
title('Waverly Hills\n Tallahassee, Florida', family='Times New Roman', 
      size=40, color='k', backgroundcolor='w', weight='bold')

<font size='5' face='Times New Roman'><b>1.13 <u>Create distance matrix</u></b></font>

In [ ]:
t1 = time.time()
All_Neigh_Dist = ntw.allneighbordistances(sourcepattern=ntw.pointpatterns['Rand_Points_SUPPLY'],
                                       destpattern=ntw.pointpatterns['Rand_Points_DEMAND'])
All_Dist_MILES = All_Neigh_Dist * float(10000/90) * 0.6214
seconds = round(time.time()-t1, 4)
print seconds, 'seconds'
print 'Supply to Demand Matrix Shape --> ', All_Dist_MILES.shape

<font size='7' face='Times New Roman'><b>2. <u>Optimize</u></b></font>

In [ ]:
t1 = time.time()
#       1. Data
# Cost Matrix
Cij = All_Dist_MILES
# Supply
Si = Si
SiSum = SiSum
# Demand
Dj = Dj
DjSum = DjSum
supply_nodes = range(len(Cij))
demand_nodes = range(len(Cij[0]))

#       2. Create Model, Set MIP Focus, Add Variables, & Update Model
m = gbp.Model(' -- The Transportation Problem -- ')
# Set MIP Focus to 2 for optimality
gbp.setParam('MIPFocus', 2)
# Create Shipping Decision Variable
shipping_var = []
for orig in supply_nodes:
    shipping_var.append([])
    for dest in demand_nodes:
        shipping_var[orig].append(m.addVar(vtype=gbp.GRB.CONTINUOUS, 
                                        obj=Cij[orig][dest], 
                                        name='x'+str(orig+1)+'_'+str(dest+1)))
# Update Model Variables
m.update()       

#       3. Set Objective Function
m.setObjective(gbp.quicksum(Cij[orig][dest]*shipping_var[orig][dest] 
                        for orig in supply_nodes for dest in demand_nodes), 
                        gbp.GRB.MINIMIZE)
                        
#       4. Add Constraints
# Add Supply Constraints
for orig in supply_nodes:
     m.addConstr(gbp.quicksum(shipping_var[dest][orig] 
                        for dest in demand_nodes) - Si[orig] == 0, 
                        'Supply_Constraint_%d' % orig)
# Add Demand Constraints
for orig in supply_nodes:
     m.addConstr(gbp.quicksum(shipping_var[orig][dest] 
                        for dest in demand_nodes) - Dj[orig] == 0, 
                        'Demand_Constraint_%d' % orig)

#       5. Optimize and Print Results
m.optimize()
t2 = time.time()-t1
print '*****************************************************************************************'
print '    | From SUPPLY Facility to DEMAND Facility x(i)_(j) shipping # of units  '
print '    |                                         ↓↓        ↓↓     ↓↓'
selected = []
for v in m.getVars():
    if v.x > 0:
        var = '%s' % v.VarName
        value = '%i' % v.x
        selected.append(v.x)
        print '    |                                       ', var, '____ Units: ' ,value
print '    | Selected Facility Locations ---------  ^^^^^^^^^^^^^^^^^^^^^^^ '
print '    | Supply/Demand Facility Combinations - ', len(selected)
print '    | Supply Units Shipped ---------------- ', SiSum
print '    | Demand Units Shipped ---------------- ', DjSum
val = m.objVal
print '    | Objective Value (total miles) ------- ', int(val)
print '    | Real Time to Optimize (sec.) -------- ', t2
print '*****************************************************************************************'
print '\nJames Gaboardi, 2015'
m.write('path.lp')